# Dọn dữ liệu

- Từ cơ sở ở phần phân tích bộ Corpus đã xử lý.
- Mục đích của phần này là từ bộ Corpus đã xử lý `./data/mental_health.csv`, ta thu được `./data/cleaned_mhc.csv` đã được dọn.
- Việc dọn dữ liệu sẽ giúp chúng ta:
    - Giảm không gian đặc trưng, nhiễu của bộ dữ liệu mà không mất mát thông tin quá lớn.
    - Bộ dữ liệu được khách quan hơn và phản ánh chính xác hơn về chủ đề được nói đến.
    - Giúp cải thiện hoặc giữ nguyên kết quả của các thuật toán học máy và giảm lượng tính toán cần thiết.

# Các bước dọn bộ Corpus đã xử lý

1. Lemmatization và loại bỏ Stop-words
2. Loại bỏ các tokens dài hơn 17 ký tự.
3. Xóa tất cả ký tự 'i' ở cuối tại mọi tokens kết thúc bằng ký tự 'i'.
4. Đưa những yếu tố có char lặp lại về 1 char (eg. soooooo -> so, dddddd -> d).
5. Loại bỏ các tokens cụ thể trên cơ sở của phần phân tích n-Grams:
- filler
- br
- nowdrink
- pee
- poo
- eve
- click
- horny
- ampxb
- monitor
- Mọi tokens bắt đầu bằng 'gt'

6. Ngoại trừ Tokens \['im' 'go', 'do'], loại bỏ mọi Tokens có độ dài nhỏ hơn 3.
7. Loại bỏ các token có tần suất nhỏ hơn 5.
8. Chuẩn hóa khoảng trắng.
9. Chỉ giữ lại các mẫu có độ dài từ 10 - 400 Tokens.
10. Loại bỏ các mẫu trống hoặc chỉ có khoảng trắng.
11. Loại bỏ các mẫu trùng.

In [1]:
# Nhập thư viện
import pandas as pd
import numpy as np
import re
import string
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Đọc bộ dữ liệu
corpus = pd.read_csv('data/processed_mhc.csv')

### 1. Lemmatization và loại bỏ Stop-words

In [2]:
# nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = text.split()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

original_text = corpus['text'].copy()
corpus['text'] = corpus['text'].apply(lemmatize_text)

affected_rows = (original_text != corpus['text']).sum()
print(f"Total number of rows affected by lemmatization: {affected_rows}")

# nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    tokens = text.split()
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(filtered_tokens)

original_text = corpus['text'].copy()
corpus['text'] = corpus['text'].apply(remove_stop_words)

affected_rows = (original_text != corpus['text']).sum()
print(f"Total number of rows affected by stop words removal: {affected_rows}")

Total number of rows affected by lemmatization: 22841
Total number of rows affected by stop words removal: 19444


### 2. Loại bỏ các tokens dài hơn 17 ký tự

In [3]:
def remove_long_tokens(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if len(token) <= 17]
    return ' '.join(cleaned_tokens)

rows_with_long_tokens = corpus[corpus['text'].str.contains(r'\b\w{18,}\b', regex=True)]
print(f"Total rows with tokens longer than 17 characters: {rows_with_long_tokens.shape[0]}")

corpus['text'] = corpus['text'].apply(remove_long_tokens)

rows_with_long_tokens_after = corpus[corpus['text'].str.contains(r'\b\w{18,}\b', regex=True)]
print(f"Number of rows with tokens longer than 17 characters after cleaning: {rows_with_long_tokens_after.shape[0]}")

Total rows with tokens longer than 17 characters: 1450
Number of rows with tokens longer than 17 characters after cleaning: 0


### 3. Xóa tất cả ký tự 'i' ở cuối tại mọi tokens kết thúc bằng ký tự 'i'

In [4]:
def find_tokens_ending_with_i(text):
    tokens = text.split()
    return [token for token in tokens if token.endswith('i')]

tokens_before_cleaning = [token for text in corpus['text'] for token in find_tokens_ending_with_i(text)]

print("Tokens ending with 'i' before cleaning:")
for token in tokens_before_cleaning[:15]:
    print(token)

def remove_trailing_i_properly(text):
    tokens = text.split()
    cleaned_tokens = [re.sub(r'i+$', '', token) for token in tokens]
    return ' '.join(cleaned_tokens)

corpus['text'] = corpus['text'].apply(remove_trailing_i_properly)

rows_with_trailing_i_after = corpus[corpus['text'].str.contains(r'\b\w*i\b')]
print(f"\nNumber of rows with tokens containing trailing 'i' after cleaning: {rows_with_trailing_i_after.shape[0]}")

Tokens ending with 'i' before cleaning:
lifei
carei
friendsi
usi
siblingsi
oni
ii
upi
chapteri
oni
bi
bi
illuminati
illuminati
tomorrowi

Number of rows with tokens containing trailing 'i' after cleaning: 0


### 4. Đưa những yếu tố có char lặp lại về 1 char

In [5]:
def handle_repeated_chars(text):
    cleaned_text = re.sub(r'(.)\1{2,}', r'\1', text)
    changes_made = sum(1 for original, cleaned in zip(text, cleaned_text) if original != cleaned)
    return cleaned_text, changes_made

original_text = corpus['text'].copy()

corpus['text'], changes_made = zip(*corpus['text'].apply(handle_repeated_chars))

total_changes = sum(changes_made)

remaining_instances = corpus[corpus['text'].str.contains(r'(.)\1{2,}', regex=True)]

print(f"Total changes made in repeated character handling: {total_changes}")
print(f"Total remaining instances with repeated characters: {remaining_instances.shape[0]}")

Total changes made in repeated character handling: 192147
Total remaining instances with repeated characters: 0


### 5. Loại bỏ các tokens cụ thể trên cơ sở của phần phân tích n-Grams

In [6]:
specific_words = ['filler', 'br', 'eve', 'pee', 'poo', 'ampxb', 'nowdrink', 'monitor', 'click', 'horny']

def remove_specific_tokens(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if token not in specific_words and not token.startswith('gt')]
    return ' '.join(cleaned_tokens)

original_text = corpus['text'].copy()

corpus['text'] = corpus['text'].apply(remove_specific_tokens)

affected_rows = (original_text != corpus['text']).sum()

print(f"Total number of rows affected: {affected_rows}")

def verify_removal(corpus, specific_words, column='text'):
    word_counts = {word: corpus[column].str.count(fr'\b{word}\b').sum() for word in specific_words}
    
    gt_count = corpus[column].str.count(r'\bgt\w+').sum()
    
    print(f"Instances found of specified words: {sum(word_counts.values())}")
    print(f"Instances found of gt-prefixed words: {gt_count}")
    
    if sum(word_counts.values()) > 0:
        for word, count in word_counts.items():
            if count > 0:
                print(f"- '{word}': {count}")

verify_removal(corpus, specific_words)

Total number of rows affected: 2521
Instances found of specified words: 0
Instances found of gt-prefixed words: 0


### 6. Ngoại trừ Tokens \['im' 'go', 'do'], loại bỏ mọi Tokens có độ dài nhỏ hơn 3

In [7]:
specific_words_to_keep = ['im', 'go', 'do']

def remove_tokens_by_length(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if len(token) > 2 or token in specific_words_to_keep]
    return ' '.join(cleaned_tokens)

original_text = corpus['text'].copy()

# Apply the cleaning process
corpus['text'] = corpus['text'].apply(remove_tokens_by_length)

affected_rows = (original_text != corpus['text']).sum()

print(f"Total number of rows affected: {affected_rows}")

remaining_instances = corpus[corpus['text'].str.split().apply(lambda x: any(len(token) <= 2 and token not in specific_words_to_keep for token in x))]

remaining_count = remaining_instances.shape[0]
print(f"Total remaining instances with tokens of 2 or less characters (excluding {specific_words_to_keep}): {remaining_count}")

Total number of rows affected: 10853
Total remaining instances with tokens of 2 or less characters (excluding ['im', 'go', 'do']): 0


### 7. Loại bỏ các token có tần suất nhỏ hơn 5

In [8]:
all_tokens = [token for text in corpus['text'] for token in text.split()]
token_freq = Counter(all_tokens)

tokens_to_remove = {token for token, freq in token_freq.items() if freq <= 4}

def remove_low_freq_tokens(text):
    tokens = text.split()
    cleaned_tokens = [token for token in tokens if token not in tokens_to_remove]
    return ' '.join(cleaned_tokens)

original_text = corpus['text'].copy()
corpus['text'] = corpus['text'].apply(remove_low_freq_tokens)

affected_rows = (original_text != corpus['text']).sum()

print(f"Total number of rows affected: {affected_rows}")

remaining_tokens = [token for text in corpus['text'] for token in text.split() if token in tokens_to_remove]

print(f"Remaining low-frequency tokens after removal: {remaining_tokens}")

Total number of rows affected: 17473
Remaining low-frequency tokens after removal: []


### 8. Chuẩn hóa khoảng trắng

In [9]:
def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

rows_with_extra_whitespace = corpus[corpus['text'].str.contains(r'\s{2,}|\t|\n', regex=True)]
print(f"Total number of rows with multiple spaces, tabs, or newlines: {rows_with_extra_whitespace.shape[0]}")

corpus['text'] = corpus['text'].apply(normalize_whitespace)

remaining_whitespace_rows = corpus[corpus['text'].str.contains(r'\s{2,}|\t|\n', regex=True)]
print(f"Number of rows with multiple spaces, tabs, or newlines after normalization: {remaining_whitespace_rows.shape[0]}")

Total number of rows with multiple spaces, tabs, or newlines: 0
Number of rows with multiple spaces, tabs, or newlines after normalization: 0


### 9. Chỉ giữ các mẫu có Total Tokens trong khoảng 10 - 400

In [10]:
num_samples_before = corpus.shape[0]
print(f"Number of samples before filtering: {num_samples_before}")

corpus = corpus[corpus['text'].apply(lambda text: len(text.split()) >= 10 and len(text.split()) <= 400)]

num_samples_after = corpus.shape[0]
print(f"Number of samples after filtering: {num_samples_after}")

Number of samples before filtering: 27969
Number of samples after filtering: 23240


### 10. Loại bỏ các điểm trống hoặc chỉ có khoảng trắng

In [11]:
total_rows_before = corpus.shape[0]
print(f"Total rows before removing empty or whitespace-only rows: {total_rows_before}")

corpus = corpus[corpus['text'].str.strip().astype(bool)]

total_rows_after = corpus.shape[0]
print(f"Total rows after removing empty or whitespace-only rows: {total_rows_after}")

rows_removed = total_rows_before - total_rows_after
print(f"Total rows removed: {rows_removed}")

Total rows before removing empty or whitespace-only rows: 23240
Total rows after removing empty or whitespace-only rows: 23240
Total rows removed: 0


### 11. Loại bỏ các điểm trùng nhau

In [12]:
before_drop_dupes = corpus.shape[0]
print(f"Total rows before removing duplicates: {before_drop_dupes}")

duplicates = corpus[corpus.duplicated(subset=['text'], keep=False)]

print("Indices of duplicate rows:")
print(duplicates.index.tolist())
print("\nContent of duplicate rows:")
print(duplicates.head(len(corpus.duplicated(subset=['text'], keep=False))))

corpus = corpus.drop_duplicates(subset=['text'], keep='first')

print(f"\nTotal rows after removing duplicates: {corpus.shape[0]}")

rows_removed = before_drop_dupes - corpus.shape[0]
print(f"Number of rows removed: {rows_removed}")

remaining_duplicates = corpus.duplicated(subset=['text']).sum()
print(f"Number of duplicate rows after cleaning: {remaining_duplicates}")

Total rows before removing duplicates: 23240
Indices of duplicate rows:
[]

Content of duplicate rows:
Empty DataFrame
Columns: [text, label]
Index: []

Total rows after removing duplicates: 23240
Number of rows removed: 0
Number of duplicate rows after cleaning: 0


In [13]:
# Lưu lại `data/cleaned_mhc.csv`
corpus.to_csv('data/cleaned_mhc.csv', index=False)